In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
import analyze_utils

# save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/'
# r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
#     save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

import pickle
r = pickle.load(open('../results/classification/r.pkl', 'rb'))

In [3]:
len(r)

30867

In [4]:
pd.set_option('display.max_colwidth', None)

top_prompts = (
    r [r['num_learned_tokens'] < 24] # Drop 24-token runs because some autoprompt runs failed (OOM).
      # .sort_values(by='prefix_train_acc', ascending=False)
      .sort_values(by='prefix_train_loss', ascending=True)
      .groupby(['num_learned_tokens', 'task_name', 'model_cls', 'seed'])
    
).first()
print(len(top_prompts))


top_prompts[['prefixes', 'reciprocal_rank', 'prefix_train_loss', 'prefix_train_acc']]

48


prefixes  \
num_learned_tokens task_name  model_cls  seed                                                                              
6                  ffb_train  autoprompt 1                                                EFFverbal EUR Thorntonshopnown   
                                         2                                             Fur resultolandgroundur augmented   
                                         3                                                      Hackmmmmajoreryitprofits   
                              iprompt    1                                                    almost neutral. However, "   
                                         2                                                              "So, a bottle of   
                                         3                                                 "Does this represent a market   
                   imdb_train autoprompt 1                                             CRIP deserves PIN SOC sling level   
                                         2                                                    as ​Overall': large points   
                                         3                               ™:Supplement Reasons****************RatingUltra   
                              iprompt    1                                                 When you watch and enjoy this   
                                         2                                                      I find this film a total   
                                         3                                                   To summarize this review! :   
                   rt_train   autoprompt 1                                         Whether{{ anotherath<|endoftext|> how   
                                         2                                    congratulations Named #SPONSOREDReport the   
                                         3                                                     wow some oneendered  very   
                              iprompt    1                                                  "not only are the characters   
                                         2                                                    Who is the author of these   
                                         3                                                   Do you agree with the above   
                   sst2_train autoprompt 1                                         \t BryceSpecificallyWASHINGTONRatedam   
                                         2                                                396 trulyCustomer echoes the "   
                                         3                                            ".Too organic appeal "… thoroughly   
                              iprompt    1                                              Can be used to describe anything   
                                         2                                         A statement that expresses a definite   
                                         3                                               Use this sentence to express an   
12                 ffb_train  autoprompt 1                    proportstals"]," AoErisome peas(" Argentina balance WININc   
                                         2                 oil feed UsingOilalyst Albert Herb Grass ling Bankingthe mild   
                                         3          izationalquartersLord quarterTableHeadperiodMON goTEXT Sylcommercial   
                              iprompt    1                                     <input> neutral> The result was due to: "   
                                         2                                A neutral sentence. Should it be: "This is the   
                                         3                                     Neutral? Hmmm. Let's think about this. It   
                   imdb_train autoprompt 1                    Luaagram RomanFaith Rockyux meets Cast Writing Rating and=   
                    

In [5]:
top_prompts.sort_values('prefix_train_acc', ascending=False).reset_index().groupby(['model_cls', 'task_name']).first()['prefixes']

model_cls   task_name 
autoprompt  ffb_train                                       Fur resultolandgroundur augmented
            imdb_train    uclear覚醒cend Koretravel NAACP curses SicAstings production received
            rt_train                                    Whether{{ anotherath<|endoftext|> how
            sst2_train                                     ".Too organic appeal "… thoroughly
iprompt     ffb_train                               <input> neutral> The result was due to: "
            imdb_train                                            'savage'<br /><br />Rating:
            rt_train                 what words would you try to add to help you express that
            sst2_train     It is clear from the sentence that all three actors have something
Name: prefixes, dtype: object

In [6]:
top_prompts.groupby(['task_name', 'model_cls']).mean()['prefix_train_acc']

task_name   model_cls 
ffb_train   autoprompt    0.692708
            iprompt       0.804688
imdb_train  autoprompt    0.888021
            iprompt       0.908854
rt_train    autoprompt    0.781250
            iprompt       0.828125
sst2_train  autoprompt    0.835938
            iprompt       0.869792
Name: prefix_train_acc, dtype: float64

In [7]:
assert r['checkpoint'].unique()[0] == "EleutherAI/gpt-j-6B"

from iprompt import prompt_classification
model = prompt_classification.create_model(r['checkpoint'].unique()[0])

2023-03-19 12:42:56.531883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 12:42:56.720508: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 12:42:56.754607: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-19 12:42:57.854843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [8]:
## Compute accuracy given correct prompt and save for each task.
import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


data = []
print('calculating accs...')
n_shots = 1
batch_size = 8

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts.reset_index().iterrows(), total=len(top_prompts)):
    verbose = False
    max_length = 512
    # if not (('ffb' in output['task_name']) or ('imdb' in output['task_name'])): continue
    if output['model_cls'] == 'autoprompt': continue
    if 'ffb' in output['task_name']: continue
    if output['num_learned_tokens'] not in [6, 12]: continue
    output['task_name'] = output['task_name'].replace('_train', '_test')
    args = argparse.Namespace(**output)
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 200 # 1_000
    print("*-*-" * 20)
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "" # tmp override
    manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=descr, multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=16, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    # print(output)
    print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=output['prefixes'], multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=16, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)

calculating accs...


  0%|          | 0/48 [00:00<?, ?it/s]

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
imdb_test
**loading data: imdb // test


Found cached dataset imdb (/home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-fb112935e367894d.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-2ef38a4eeeffa693.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-2c774f3564e01edb.arrow


  0%|          | 0/13 [00:00<?, ?it/s]

...

Found cached dataset sst2 (/home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-48129d1559bb651f.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-93efb9f1d3e163f8.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-3fb54b696b14ee69.arrow


  0%|          | 0/13 [00:00<?, ?it/s]

	 || 75.5%


  0%|          | 0/13 [00:00<?, ?it/s]

	 It is clear from the sentence that all three actors have something || 82.5%


In [12]:
df_with_acc = pd.DataFrame(data)
# df_with_acc[['task_name', 'model_cls', 'seed', 'prefixes', 'prefix_train_acc', 'iprompt_acc', 'manual_acc']]
df_with_acc[df_with_acc['num_learned_tokens'] < 24].groupby(['num_learned_tokens', 'task_name', 'model_cls']).mean()[['iprompt_acc', 'manual_acc']]

iprompt_acc  manual_acc
num_learned_tokens task_name model_cls                         
6                  imdb_test iprompt      86.833333   80.833333
                   rt_test   iprompt      72.833333   73.500000
                   sst2_test iprompt      80.166667   75.500000
12                 imdb_test iprompt      93.833333   82.500000
                   rt_test   iprompt      83.166667   73.500000
                   sst2_test iprompt      82.833333   75.500000

In [13]:
final_prompts_df = df_with_acc[df_with_acc['num_learned_tokens'] < 24]
pd.set_option('display.max_rows', None)

final_prompts_df[['num_learned_tokens', 'model_cls', 'task_name', 'prefixes', 'iprompt_acc']]

,num_learned_tokens,model_cls,task_name,prefixes,iprompt_acc
9,6,iprompt,imdb_test,When you watch and enjoy this,87.0
10,6,iprompt,imdb_test,I find this film a total,92.0
11,6,iprompt,imdb_test,To summarize this review! :,81.5
15,6,iprompt,rt_test,"""not only are the characters",78.0
16,6,iprompt,rt_test,Who is the author of these,68.0
17,6,iprompt,rt_test,Do you agree with the above,72.5
21,6,iprompt,sst2_test,Can be used to describe anything,81.0
22,6,iprompt,sst2_test,A statement that expresses a definite,78.5
23,6,iprompt,sst2_test,Use this sentence to express an,81.0
33,12,iprompt,imdb_test,This movie needs to be put up on my profile as my,92.0


In [ ]:
final_prompts_df.groupby(['model_cls', 'task_name']).mean()[['iprompt_acc', 'manual_acc']]

In [ ]:
final_prompts_df.groupby(['model_cls', 'task_name']).sem()[['iprompt_acc', 'manual_acc']]

In [ ]:
## Promptsource GPT-J Accuracy
from iprompt import prompt_classification
from tqdm.notebook import tqdm
from iprompt.data import get_data

PROMPTSOURCE_PROMPTS = [
    ' The movie review in negative/positive sentiment is:',
    ###############################################################
    ' Does the previous input have a positive or negative sentiment?',
    ' Was that review positive or negative?',
    ' The sentiment expressed for the movie is',
    ' What is the sentiment expressed by the reviewer for the movie?',
    ' How does the viewer feel about the movie?',
    ' Is this review positive or negative?',
    ' What is the sentiment expressed in this text?',
    ' What sentiment does the writer express for the movie?',
]

# Financial phrasebank has a neutral class
PROMPTSOURCE_PROMPTS_NEUTRAL = [
    ' Does the previous input have a positive, neutral, or negative sentiment?',
    ' Was that input positive, neutral, or negative?',
    ' The sentiment expressed in the headline is',
    ' What is the sentiment expressed by the reviewer for the movie?',
    ' How does the author of the news headline feel?',
    ' Is this news headline positive, neutral, or negative?',
    ' What is the sentiment expressed in this text?',
    ' What sentiment does the writer express?',
]


prompt_data = []
task_names = r.reset_index()['task_name'].unique().tolist()
for task_name in tqdm(task_names):
    verbose = False
    max_length = 128
    prompts = PROMPTSOURCE_PROMPTS_NEUTRAL if ('ffb' in task_name) else PROMPTSOURCE_PROMPTS
    for _idx, _prompt in enumerate(prompts):
        output = {}
        output['task_name'] = task_name.replace('_train', '_test')
        train_split_frac = 1.0 # take 100% of test set
        max_dset_size = 1_000
        (dset, __dset_test), check_answer_func, descr = get_data(
            output['task_name'], n_shots=1, train_split_frac=1.0,
            max_dset_size=max_dset_size, template_num_task_phrasing=0,
        )
    #     ####   human-written prompt   ####
        loss, acc = prompt_classification.test_model_on_task_with_prefix(
            dset=dset, model=model, prefix=_prompt, multi_token=False, verbose=verbose,
            max_length=max_length, batch_size=64, tqdm_notebook=True,
            restrict_to_valid_answers=True,
            prefix_before_input=False,
        )
        print(f'\t{_prompt} || {acc:.1f}%')
        ####
        output['prompt_idx'] = _idx
        output['acc'] = acc
        prompt_data.append(output)

In [ ]:
human_prompt_df = pd.DataFrame(prompt_data)
human_prompt_df.groupby('task_name').mean()

In [ ]:
human_prompt_df = pd.DataFrame(prompt_data)
human_prompt_df.groupby('task_name').sem()

In [ ]:
top_prompts.reset_index()[
    (top_prompts.reset_index()['task_name'] == 'rt_train') & 
    (top_prompts.reset_index()['model_cls'] == 'iprompt') & 
    (top_prompts.reset_index()['num_learned_tokens'] == 6)
]['prefixes']

In [ ]:
PROMPTSOURCE_PROMPTS[0]

In [ ]:
(dset, __dset_test), check_answer_func, descr = get_data(
    'rt_test', n_shots=1, train_split_frac=1.0,
    max_dset_size=100, template_num_task_phrasing=0,
)
print('dset[0] =', dset[0])
small_max_length = 64
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix="", multi_token=False, verbose=False,
    max_length=small_max_length, batch_size=64, tqdm_notebook=True,
    restrict_to_valid_answers=True,
    prefix_before_input=False,
)
print("blank acc:", acc)


loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=' The movie review in negative/positive sentiment is:', multi_token=False, verbose=verbose,
    max_length=small_max_length, batch_size=64, tqdm_notebook=True,
    restrict_to_valid_answers=True,
    prefix_before_input=False,
)
print("pref acc:", acc)


loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix='Do you agree with the above', multi_token=False, verbose=verbose,
    max_length=small_max_length, batch_size=64, tqdm_notebook=True,
    restrict_to_valid_answers=True,
    prefix_before_input=False,
)
print("iprompt acc:", acc)

In [ ]:
human_prompt_df = pd.DataFrame(prompt_data)
human_prompt_df.groupby('task_name').mean()

In [ ]:
len(PROMPTSOURCE_PROMPTS_NEUTRAL)

In [ ]:
def get_prompt_from_hpdf_row(row):
    idx = row['prompt_idx']
    prompts = PROMPTSOURCE_PROMPTS_NEUTRAL if ('ffb' in row['task_name']) else PROMPTSOURCE_PROMPTS
    return prompts[idx]
    
human_prompt_df['prompt'] = human_prompt_df.apply(get_prompt_from_hpdf_row, axis=1)
(human_prompt_df.sort_values(by='acc', ascending=False).groupby('task_name').first())

In [ ]:
human_prompt_df.groupby('task_name').sem()

# Loading with PromptSource

In [ ]:
import promptsource
import promptsource.templates

imdb_prompts = promptsource.templates.DatasetTemplates('rotten_tomatoes')
# ffb_prompts = promptsource.templates.DatasetTemplates('financial_phrasebank', 'sentences_allagree')

pos_input = { "text": "\"What a wonderful film :) \"", "label": 1 }
neg_input = { "text": "\"This movie sucks!\"", "label": 0 }

for tn in imdb_prompts.all_template_names:
    print(tn)
    print('\t [+]', imdb_prompts[tn].apply(pos_input))
    print('\t [-]', imdb_prompts[tn].apply(neg_input))
    print()

In [ ]:
imdb_prompts["Movie Expressed Sentiment"].apply({ "text": "This movie sucks!" })

## Sentiment loss

In [ ]:
## Plot losses


save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/'
r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
    save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

In [ ]:
top_prompts = (
    r
      # .sort_values(by='prefix_train_acc', ascending=False)
      # .sort_values(by='prefix_train_loss', ascending=True)
      .groupby(['num_learned_tokens', 'task_name', 'model_cls', 'seed'])
    
).first().reset_index()
len(top_prompts)

In [ ]:
import numpy as np
max_loss_len = max([len(L) for L in all_losses])

all_losses_padded = []
for L in all_losses:
    last_val = L[-1]
    all_losses_padded.append(
        L + [last_val] * (max_loss_len - len(L))
    )
all_losses_padded = np.array(all_losses_padded)
len(all_losses_padded)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import math

COLORS = sns.color_palette()

BATCH_SIZE = 16
X_AXIS = np.arange(all_losses_padded.shape[1]) 

#############################################

tasks = top_prompts['task_name'].unique()

#############################################
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(12, 8),
                        layout="constrained")

for idx, task in enumerate(tasks):
    for m_idx, model_cls in enumerate(['iprompt', 'autoprompt']):
        MASK = (
              (top_prompts['model_cls'] == model_cls)
            & (top_prompts['task_name'] == task)
            & (top_prompts['num_learned_tokens'] < 24)
        )
        
        i = idx // 2
        j = idx % 2
        
        axs[i,j].plot(X_AXIS, all_losses_padded[MASK].mean(axis=0), color=COLORS[m_idx], label=model_cls, linewidth=3)
    axs[i,j].set_title(task)


#############################################

plt.title('iPrompt Loss')
plt.xlabel('Number of iterations')
plt.ylabel('NLL')
plt.legend()
plt.yscale('log')


pdf_filename = 'sentiment_loss_curves.pdf'
plt.tight_layout()
plt.savefig(pdf_filename, dpi=300)